In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Create a new set of 2,000 random latitudes and longitudes
coordinates = list(lat_lngs)

In [4]:
# Now find the cities nearby these coordinates and create a list 

from citipy import citipy

cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # adding to list only if not already added 
    if city not in cities:
        cities.append(city)

len(cities)

806

In [5]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
from datetime import datetime

In [7]:
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        req = requests.get(city_url)
        city_weather =req.json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        city_desc = city_weather["weather"][0]["description"]
        # Append the City information into city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                         "Weather Desc": city_desc})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | lorengau
Processing Record 2 of Set 1 | hamilton
Processing Record 3 of Set 1 | torbay
Processing Record 4 of Set 1 | kavieng
Processing Record 5 of Set 1 | nusaybin
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | natal
Processing Record 8 of Set 1 | barrow
Processing Record 9 of Set 1 | faanui
Processing Record 10 of Set 1 | javanrud
Processing Record 11 of Set 1 | bluff
Processing Record 12 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 13 of Set 1 | sioux lookout
Processing Record 14 of Set 1 | babanusah
City not found. Skipping...
Processing Record 15 of Set 1 | port alfred
Processing Record 16 of Set 1 | husavik
Processing Record 17 of Set 1 | wanlaweyn
Processing Record 18 of Set 1 | saint-philippe
Processing Record 19 of Set 1 | kapaa
Processing Record 20 of Set 1 | sao joao da barra
Processing Record 21 of Set 1 | fukue
Processing Record 22 o

In [8]:
city_data_df = pd.DataFrame(city_data)

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
0,Lorengau,PG,-2.0226,147.2712,81.86,76,100,8.50,overcast clouds
1,Hamilton,US,39.1834,-84.5333,48.72,51,0,4.00,clear sky
2,Torbay,CA,47.6666,-52.7314,19.76,72,20,5.75,few clouds
3,Kavieng,PG,-2.5744,150.7967,82.72,75,95,9.55,overcast clouds
4,Nusaybin,TR,37.0778,41.2178,51.39,93,75,13.80,mist


In [9]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")